# Tutorial on using Propnet

The following is a tutorial designed to give a base overview of the classes and constructs used in the Propnet project. For each class an example of its construction and base usage is provided.

# Defining a Property Network: Propnet

A Propnet object tells us all about the property types and models currently available for use.

The mappings contained in this object define an interconnected network of materials properties. In this form, the Propnet object can be used to enumerate and analyze links between differnet materials properties.

In [8]:
from propnet import Propnet

In [9]:
p = Propnet()

'ascii' codec can't decode byte 0xc3 in position 626: ordinal not in range(128)
'ascii' codec can't decode byte 0xcf in position 176: ordinal not in range(128)
'ascii' codec can't decode byte 0xce in position 258: ordinal not in range(128)
'ascii' codec can't decode byte 0xc2 in position 529: ordinal not in range(128)


You can print Propnet to see the property types and models it supports.

In [10]:
print(p)

Propnet Graph

Property Types:
	 Crystallographic structure (oxidation-state decorated)
		 Decorate crystal structure with oxidation state
	 Is Metallic
		 Wiedemann-Franz Law
		 Metallic Classifier
	 Crystallographic structure
	 Formula
	 Crystal Prototype
	 Elastic tensor (in Voigt notation)
	 Interplanar Spacing
	 Final Energy
	 Goldschmidt tolerance factor
	 Thermal conductivity
		 Clarke thermal conductivity
	 Refractive index
		 Optical Reflectance from the complex refractive index
		 Refractive index, relative permeability and permittivity
	 Ionic radius of B site in perovskite
		 Perovskite Classifier
	 Absorption Coefficient
		 Optical Absorbance from the absorption coefficient, thickness and reflectance
		 Optical Absorption coefficient alpha from extinction coeff and wavelength
	 P-wave modulus
	 Extinction Coefficient
		 Optical Reflectance from the complex refractive index
		 Optical Absorption coefficient alpha from extinction coeff and wavelength
	 Thermal conductivity, 

Or you can iterate over the graph to see how it works behind the scenes.

In [11]:
for n in p.graph.nodes():
    print(n)

PropnetNode(node_type=<PropnetNodeType.SymbolType: 2>, node_value=<SymbolType.structure_oxi: <propnet.core.symbols.SymbolMetadata object at 0x113ae0de0>>)
PropnetNode(node_type=<PropnetNodeType.SymbolType: 2>, node_value=<SymbolType.is_metallic: <propnet.core.symbols.SymbolMetadata object at 0x113ae0f48>>)
PropnetNode(node_type=<PropnetNodeType.SymbolType: 2>, node_value=<SymbolType.structure: <propnet.core.symbols.SymbolMetadata object at 0x113b6e048>>)
PropnetNode(node_type=<PropnetNodeType.SymbolType: 2>, node_value=<SymbolType.pretty_formula: <propnet.core.symbols.SymbolMetadata object at 0x113b6e0c0>>)
PropnetNode(node_type=<PropnetNodeType.SymbolType: 2>, node_value=<SymbolType.prototype: <propnet.core.symbols.SymbolMetadata object at 0x113b6e138>>)
PropnetNode(node_type=<PropnetNodeType.SymbolType: 2>, node_value=<SymbolType.elastic_tensor_voigt: <propnet.core.symbols.SymbolMetadata object at 0x113b6e1b0>>)
PropnetNode(node_type=<PropnetNodeType.SymbolType: 2>, node_value=<Symbo

# Defining a Symbol or Property

A SymbolType object is used to represent types of properties (such as Young's Modulus) or conditions (such as Temperature).
- All SymbolType objects are accessible in a global SymbolType variable.
- Various metadata for each SymbolType can be accessed as shown below.

In [20]:
from propnet.symbols import SymbolType

In [21]:
symbol_type_object = SymbolType['youngs_modulus'].value

In [22]:
symbol_type_object.as_dict()

{'@class': 'SymbolMetadata',
 '@module': 'propnet.core.symbols',
 'comment': '',
 'dimension': 1,
 'display_names': ["Young's modulus", 'Elastic modulus'],
 'display_symbols': ['E'],
 'name': 'youngs_modulus',
 'test_value': 130.0,
 'type': 'property',
 'units': <Quantity(1.0, 'gigapascal')>}

A Symbol object is used to represent values of properties (such as Young's Modulus = 200GPa) or conditions (such as temperature = 300K).

- All Symbol objects have a SymbolType giving the type of property represented by the value.
- All Symbol objects must be created at runtime by specifying a value during instantiation.
- All Symbol objects have a list of strings called "tags" used to further label the property.

In [23]:
from propnet.symbols import SymbolType
from propnet import Symbol

In [24]:
youngs_modulus_type = SymbolType['youngs_modulus'].value
steel_youngs_modulus = Symbol(youngs_modulus_type, 200, ['mild steel'])

In [25]:
steel_youngs_modulus.as_dict()

{'@class': 'Symbol',
 '@module': 'propnet.core.symbols',
 'provenance': None,
 'tags': ['mild steel'],
 'type': {'@class': 'SymbolMetadata',
  '@module': 'propnet.core.symbols',
  'comment': '',
  'dimension': 1,
  'display_names': ["Young's modulus", 'Elastic modulus'],
  'display_symbols': ['E'],
  'name': 'youngs_modulus',
  'test_value': 130.0,
  'type': 'property',
  'units': <Quantity(1.0, 'gigapascal')>},
 'value': 200}

# Defining a Material

A Material object is used to represent a collection of information known about a given material.

When it is first created it has no information; however, properties can be added to the material one-by-one.

In [26]:
from propnet import Material
from propnet.symbols import SymbolType
from propnet import Symbol

In [27]:
mild_steel = Material()
youngs_modulus = Symbol(SymbolType['youngs_modulus'], 200, [])
mild_steel.add_property(youngs_modulus)

In [28]:
print(mild_steel)

Material: 0aae621a-318d-481c-9891-b2b1acbc07da
	youngs_modulus:	200



# Combining Models, Materials, and Symbols

As illustrated, a Propnet object contains information for connecting many different models and symbol types. This forms an abstract web of interconnected variables without any quantities specified.

On the other hand, a Material object represents a grouping of values for different variables. These are represented as a collection of Symbol objects identified with the material.

At runtime, a single Propnet object can be combined with one or more Material objects. This procedure allows values to be plugged in to variables. Assuming the required inputs for a model all have values, the Propnet object can then dynamically predict the values for the output variables of the model.

In [38]:
## Setting up the example:
from propnet import Propnet
p = Propnet()

silica = Material()
refractive_index = Symbol(SymbolType['refractive_index'], 1.458, [])
relative_permittivity = Symbol(SymbolType['relative_permittivity'], 3.9, [])

silica.add_property(refractive_index)
silica.add_property(relative_permittivity)

p.add_material(silica)

'ascii' codec can't decode byte 0xc3 in position 626: ordinal not in range(128)
'ascii' codec can't decode byte 0xcf in position 176: ordinal not in range(128)
'ascii' codec can't decode byte 0xce in position 258: ordinal not in range(128)
'ascii' codec can't decode byte 0xc2 in position 529: ordinal not in range(128)


Propnet can now examine the input values and identify if any models can be used to derive additional properties.

In this example, we've provided the relative permittivity and index of refraction of silica. Thus, using the canonical relationship from electromagnetism, we expect Propnet to properly derive the relative permeability.

Re-examining the material object previously created, a new Symbol object, the relative permeability, is now associated with that material.

In [39]:
p.evaluate(material=silica)
print(silica)

Material: 7f12bd60-4f7c-4c50-a606-f386258bc0fe
	refractive_index:	1.458
	relative_permittivity:	3.9
	relative_permeability:	0.545067692307692



# Working with Models

A Model object is used to represent a relationship between different materials property variables. This object can be directly manipulated and stores relavent metadata available as direct attributes.

- All Models are imported as classes at runtime.
- A Model class must be instantiated to be used at runtime.

In [16]:
from propnet.models import *
model = RefractiveIndexfromRelPerm()
print(model.description)
print(model.name)
print(model.equations)


The refractive index gives the factor by which the speed of light is reduced in a medium.

Likewise, modeling the induced magnetic and electric dipoles as linear within a material,
a relative spatial electrical permittivity and relative spatial magnetic permeability
arise from consideration of the total electrical and magnetic fields.

From the Maxwell Relations, the index of refraction is equal to the geometric mean  of the
relative permittivity and the relative permeability.

RefractiveIndexfromRelPerm
['n - sqrt(Ur*Er)']


The Model class is a generally-defined interface, and subclasses may alter many aspects of its underlying functionality.


Most Model objects will contain equations, symbols, and connections attributes. These define the core functionality of the model:

The equations attribute will contain a list of sympy-parsable expressions. These expressions imply trivial equations such that the expression is equal to zero.

The symbols attribute map the symbols used in the equations to Symbol_Type objects used in the Property Network.

The connections attribute shows what outputs can be generated from a set of inputs.

In [17]:
print(model.symbol_mapping)
print(model.equations)
print(model.connections)

{'Er': 'relative_permeability', 'Ur': 'relative_permittivity', 'n': 'refractive_index'}
['n - sqrt(Ur*Er)']
[{'inputs': ['Ur', 'Er'], 'outputs': 'n'}, {'inputs': ['Er', 'n'], 'outputs': 'Ur'}, {'inputs': ['Ur', 'n'], 'outputs': 'Er'}]


A Model can be evaluated to generate outputs if given a complete set of inputs.

Given the relative permeability and permittivity, the Refractive Index From Relative Permeability model can correctly calculate the index of refraction.

In [18]:
model.evaluate({'Ur': 0.54, 'Er': 3.9})

{'n': 1.45120639469374, 'successful': True}

# Coming Soon...

In [41]:
from pint import UnitRegistry

In [42]:
ureg = UnitRegistry()

In [43]:
ureg.parse_expression("gigapascal").to_tuple()

(1, (('gigapascal', 1.0),))

In [44]:
node_list = list(p.graph.nodes)

In [45]:
from enum import Enum

In [46]:
idx = 10
print(node_list[idx])
print(type(node_list[idx]))
if isinstance(node_list[idx], Enum):
    print(node_list[idx].value)
    print(type(node_list[idx].value))

PropnetNode(node_type=<PropnetNodeType.SymbolType: 2>, node_value=<SymbolType.elastic_tensor_voigt: <propnet.core.symbols.SymbolMetadata object at 0x10524f318>>)
<class 'propnet.core.graph.PropnetNode'>


In [47]:
my_material = Material.from_mpid("mp-12345")
p.add_material(my_material)

AttributeError: type object 'Material' has no attribute 'from_mpid'

# Defining a Model with Constraints

In [48]:
class MySampleModel(AbstractModel):
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-48-94a310fbd9fe>, line 3)